In [ ]:
%load_ext autoreload
%autoreload 2

import zarr
import xarray as xr
import os

In [ ]:
# rechunked version
mapper = zarr.storage.ABSStore(
    "carbonplan-scratch",
    prefix="rechunker/terraclimate/target.zarr/",
    account_name="carbonplan",
    account_key=os.environ["BLOB_ACCOUNT_KEY"],
)

ds_conus = xr.open_zarr(mapper, consolidated=True)

In [ ]:
from carbonplan.data import cat

ds_global = cat.terraclimate.raw_terraclimate.to_dask()

In [ ]:
dem = xr.open_rasterio(
    "https://storage.googleapis.com/carbonplan-data/"
    "processed/edna/conus/4000m/raster.tif"
).load()
dem = dem.where(dem > -1000).squeeze(drop=True)
ds_conus["dem"] = dem

In [ ]:
dbr = xr.open_rasterio(
    "/home/jovyan/cmip6-downscaling/cmip6_downscaling/" "data/rockdepm.tif"
)
dbr = dbr.where(dbr != 255).squeeze(drop=True)
ds_conus["dbr"] = dbr.variable

In [ ]:
from metpy.calc import dewpoint
from metpy.units import units
import numpy as np
from cmip6_downscaling.disagg import terraclimate

In [ ]:
# somewhere in idaho
# df = ds_conus.isel(x=200, y=200).squeeze(drop=True).to_dataframe()

# kitsap peninsula
# df = ds_conus.sel(x=-2e6, y=3e6, method='nearest').squeeze(drop=True).to_dataframe()

# west slopes colorado rockies
df = (
    ds_conus.sel(x=-1e6, y=2e6, method="nearest")
    .squeeze(drop=True)
    .to_dataframe()
)

# southeast
# df = ds_conus.sel(x=1e6, y=1e6, method='nearest').squeeze(drop=True).to_dataframe()

In [ ]:
awc = df.soil.max()  # we use this until we get john's data
awc = xr.open_rasterio(
    "/home/jovyan/cmip6-downscaling/cmip6_downscaling/" "data/awc_250.tif"
).load()
awc = awc.where(awc != 255).squeeze(drop=True)
ds_conus["awc"] = awc.variable

## Soil water capacity derivation for Terraclimate

The WBM requires data on the plant extractable soil water capacity. We used
extractable soil water storage capacity data at a 0.5° grid from Wang-Erlandsson
et al.32 that were derived from estimates of satellite based evaporation, ET0,
and precipitation. Wang-Erlandsson et al.32 provided estimates for varying
return intervals. _We used the 2-year period_ given our use of ET0, as it best
matched with evapotranspiration variability in grassland biomes, corresponding
with our use of a reference grass surface in the WBM. To adhere to the spatial
resolution of TerraClimate, _we first extrapolated data using a 0.5° buffer from
existing data using the 8 nearest neighbors, and then bilinearly interpolated
plant extractable soil water from its native 0.5° grid. We set a lower bound on
plant extractable soil water of 10 mm and set a default value of 50 mm for
places void of data._


In [ ]:
swc_chirps = xr.open_rasterio(
    "/home/jovyan/cmip6-downscaling/cmip6_downscaling/"
    "data/sr_chirps_2yrs.asc"
)
swc_cru = xr.open_rasterio(
    "/home/jovyan/cmip6-downscaling/cmip6_downscaling/" "data/sr_cru_2yrs.asc"
)
swc = swc_cru.where(swc_cru != -9999).squeeze(drop=True)

In [ ]:
df_global = (
    ds_global.sel(lat=df.lat[0], lon=df.lon[0], method="nearest")
    .squeeze(drop=True)
    .to_dataframe()
)

In [ ]:
df["awc"] = swc.sel(y=df.lat[0], x=df.lon[0], method="nearest").values

In [ ]:
df.update(df_global)

In [ ]:
df["tmean"] = (df.tmax + df.tmin) / 2
df["tdew"] = np.asarray(dewpoint(df["vap"].values * units.pascal * 1000))
df.head()

In [ ]:
%time
WM2_TO_MGM2D = 86400 / 1e6

import pandas as pd
import matplotlib.pyplot as plt

df_v2 = pd.DataFrame(index=df.index, columns=['snowpack', 'h2o_input', 
                                              'albedo', 'pet', 'aet', 
                                              'soil', 'runoff'])
# df['awc'] *= 25.4 * 10

snowpack_prev = 0.
tmean_prev = df['tmean'][0]
soil_prev = 0 # df['awc'][0]
count=0
for i, row in df.iterrows():
    out = terraclimate.snowmod(
        row['tmean'],
        row['ppt'],
        radiation=row['srad'] * WM2_TO_MGM2D,
        snowpack_prev=snowpack_prev)
    
    out['pet'] = terraclimate.monthly_PET(
        row['srad'] * WM2_TO_MGM2D,
        row['tmax'],
        row['tmin'],
        row['ws'],
        row['tdew'],
        tmean_prev,
        row['lat'],
        row['dem'],
        i.month - 1,
    ) * out['mfsnow']
        
    out.update(terraclimate.hydromod(
            row['tmean'],
            row['ppt'],
            out['pet'],
            row['awc'],
            soil_prev,
            snowpack_prev,
#         instead of running snowmod twice just grab it from earlier
#         run of snowmod
            out['mfsnow']))
    df_v2.loc[i] = out
    tmean_prev = row['tmean']
    snowpack_prev = out['snowpack']
    soil_prev = out['soil']

# runoff


In [ ]:
# kitsap
var = "runoff"
s = slice(-48, None)
df.q[s].plot(label="v1")
df_v2[var][s].plot(label="v2")
plt.legend()

In [ ]:
# colorado rockies
var = "runoff"
s = slice(-48, None)
df.q[s].plot(label="v1")
df_v2[var][s].plot(label="v2")
plt.legend()

In [ ]:
# southeast
var = "runoff"
s = slice(-48, None)
df.q[s].plot(label="v1")
df_v2[var][s].plot(label="v2")
plt.legend()

# aet and pet


In [ ]:
# kitsap
s = slice(-48, None)
df.pet[s].plot(label="pet-v1")

df_v2["pet"][s].plot(label="pet-v2")
df.aet[s].plot(label="v1")
df_v2.aet[s].plot(label="v2")
plt.legend()

In [ ]:
# colorado rockies
s = slice(-48, None)
df.pet[s].plot(label="pet-v1")

df_v2["pet"][s].plot(label="pet-v2")
# df_v2.et0[s].plot(label='et0-v2')
df.aet[s].plot(label="aet-v1")
df_v2.aet[s].plot(label="aet-v2")
plt.ylabel("et")
plt.legend()

In [ ]:
# somewhere in idaho
s = slice(-48, None)
df.pet[s].plot(label="pet-v1")
df_v2["pet"][s].plot(label="pet-v2")
df.aet[s].plot(label="v1")
df_v2.aet[s].plot(label="v2")
plt.legend()

# soil


In [ ]:
# southeast
s = slice(-72, None)
df.soil[s].plot(label="v1")
df_v2.soil[s].plot(label="v2")
plt.legend()

In [ ]:
# west slopes colorado rockies
s = slice(-72, None)
df.soil[s].plot(label="v1")
df_v2.soil[s].plot(label="v2")
plt.legend()

In [ ]:
# kitsap
s = slice(-72, None)
df.soil[s].plot(label="v1")
df_v2.soil[s].plot(label="v2")
plt.legend()

# swe


In [ ]:
# colorado rockies
s = slice(-72, None)
df.swe[s].plot(label="v1")
df_v2.snowpack[s].plot(label="v2")
plt.legend()